In [1]:
# import osmnx as ox
import pandas as pd
import geopandas as gpd
import json
from io import StringIO
import warnings
import sys
import os
warnings.filterwarnings("ignore")
sys.stderr = open(os.devnull, 'w')

# local crs
local_crs = 32636

In [2]:
from popframe.models.region import Region

region_model = Region.from_pickle('data/model_data/region_noter.pickle')

In [3]:
from popframe.method.territory_evaluation import TerritoryEvaluation
from popframe.models.region import Region

evaluation = TerritoryEvaluation(region=region_model)

# Пример данных
geojson_data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              29.982879431084967,
              59.363554752593245
            ],
            [
              29.982879431084967,
              59.322083801173534
            ],
            [
              30.109075699649765,
              59.322083801173534
            ],
            [
              30.109075699649765,
              59.363554752593245
            ],
            [
              29.982879431084967,
              59.363554752593245
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

result = evaluation.evaluate_territory_location(territories=geojson_data)
print(result)

[{'territory': 'None', 'score': 3, 'interpretation': 'территория находится внутри или непосредственной близости населенного пункта уровня Большое сельское поселение', 'closest_settlement': 'Сиверский', 'closest_settlement1': None, 'closest_settlement2': None}]


In [4]:
from popframe.method.territory_evaluation import TerritoryEvaluation

evaluation = TerritoryEvaluation(region=region_model)

# Пример использования функции
results = evaluation.evaluate_territory_location(territories = None)

for result in results:
    if result['closest_settlement']:
        print(f"Территория: {result['territory']}, Оценка: {result['score']}, Интерпретация: {result['interpretation']}, Ближайший населённый пункт: {result['closest_settlement']}")
    else:
        print(f"Территория: {result['territory']}, Оценка: {result['score']}, Интерпретация: {result['interpretation']}. Территория находится между населенными пунктами: {result['closest_settlement1']} и {result['closest_settlement2']}.")

Территория: Шиссельбург, Оценка: 5, Интерпретация: территория находится внутри или непосредственной близости населенного пункта уровня Малый город, Ближайший населённый пункт: Шлиссельбург


In [5]:
results

[{'territory': 'Шиссельбург',
  'score': 5,
  'interpretation': 'территория находится внутри или непосредственной близости населенного пункта уровня Малый город',
  'closest_settlement': 'Шлиссельбург',
  'closest_settlement1': None,
  'closest_settlement2': None}]